Fingerprint would be help to answer the question about how computer represents molecules? and how to Quickly compare two complicated structures?

If the fingerprint matches, the two structures are not guaranteed to be exactly the same.

But should the fingerprint does not even match, the two structures are for sure different.

In [2]:
from rdkit import Chem

sm = "C1=CC=CC=C1"
mol = Chem.MolFromSmiles(sm)

The MACCS FP is a standardised 166-bit vector that each bit represents a given chemical feature. [Link](https://github.com/rdkit/rdkit/blob/master/rdkit/Chem/MACCSkeys.py).

In [3]:
from rdkit.Chem import MACCSkeys
fp = MACCSkeys.GenMACCSKeys(mol)

In [5]:
print(fp) #see, it is a bit vector!

In [6]:
fp.ToBitString()

'00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011010'

if we take a look at the documentation on rdkit - MACCSkeys, check what is the bit equals 1.

We can find position 162, 163, 165 are 1, the rest are all 0.

What does that mean?

162 - Aromatic
163 - 6-membered ring
165 - Ring

It makes sense because our imput smiles is a benzene.

In [7]:
#Above, I count the 1 from the end of the string (and because I know it has 166 positions, so it's easy to find the last few 1s)

#there are a few functions to automatically pick out the 1 and 0 position

In [8]:
print(f"Total number of bits: {fp.GetNumBits()}") # get the total number of bits
print(f"Total number of 0 - off bits: {fp.GetNumOffBits()}")
print(f"Total number of 1 - on bits: {fp.GetNumOnBits()}")

Total number of bits: 167
Total number of 0 - off bits: 164
Total number of 1 - on bits: 3


In [9]:
fp_bit_string = fp.ToBitString()

In [12]:
len(fp_bit_string) #the first bit is set as 0 in a 167-bit string so as to keep the index starting at 1

167

In [21]:
all_on_bit = []

for i in range(len(fp_bit_string)):
    on_bit = {}
    this_bit = int(fp_bit_string[i])
    if this_bit == 1:
        on_bit["idx"] = i
        on_bit["chem_mean"] = MACCSkeys.smartsPatts[i]
        all_on_bit.append(on_bit)

print(all_on_bit)
        

[{'idx': 162, 'chem_mean': ('a', 0)}, {'idx': 163, 'chem_mean': ('*1~*~*~*~*~*~1', 0)}, {'idx': 165, 'chem_mean': ('[R]', 0)}]


Morgan - circular fp, radius-based

cf. ECFP and FCFP, diameter-based

In [25]:
from rdkit.Chem import AllChem
fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits = 1024).ToBitString() #2 means two atoms from the centre heavy atom, the radius
print(len(fp)) #determined by the nBits
print(fp)

1024
00000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [35]:
#a test to first get smiles from synonyms, then get the FCFP from rdkit
import requests
import time

url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/diphenhydramine/property/IsomericSMILES/txt"
res = requests.get(url)

In [36]:
print(res.text)

CN(C)CCOC(C1=CC=CC=C1)C2=CC=CC=C2



In [37]:
sm = res.text.rstrip()

In [38]:
mol = Chem.MolFromSmiles(sm)

In [39]:
fp = AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits = 512).ToBitString()
print(fp)

01000000000001000000000000000000010000000000010000000000000000001000000000100000100000000000000100000000000000000000000000000000000000000001100000000000000000000000000000000001000001010000000100000000000000000000001000000000010000000000000000000000000000000000000000010000001000000000000000000000000000100001000000001000000000000000000001000100000000000000100000100000010000000000000000000100000010000000000100010001000000000000000000000000000000000000000000000000000000000000000000000010000000000000100000000000


topological fingreprint (rdkit fp), enumerate through a range of length of pathway